In [1]:
import matlab.engine
import pandas
from keras.models import load_model
import random
import logging

logging.basicConfig(level=logging.INFO)

Import essential local libraries

In [2]:
%run read_data_and_prepare_to_training.ipynb
%run load_models.ipynb

ModuleNotFoundError: No module named 'tsai'

ModuleNotFoundError: No module named 'tsai'

## Parameters of failures
Provide in the next cell parameters of failures which you want to predict:
- **`SDrift`** determines the drift offset of the sensor <-1 ; 1>
- **`SGain`** controls magnitude of the disturbance of the gear tooth fault<-2 ; 0>
- **`ShaftWear`** determines subsystem variants which changes the shaft damping . The selected variant is controlled by the model variable ShaftWear, note that ShaftWear=0 implies no shaft fault.

In [ ]:
SDrift = 0
SGain = -1
ShaftWear = 0

In [ ]:
eng = matlab.engine.start_matlab()
correct = eng.SimulateTransmissionCasing(SDrift,SGain,ShaftWear)
eng.exit()

if not correct:
    logger.error("Execution of simulation was incorrect")
    raise SystemExit("Simulink error!")

### Read simulation data from file 

In [ ]:
simulation_data = pandas.read_parquet("SimulationData.parquet").to_pandas()
simulation_data

### Simulation parameters required for converting data

In [ ]:
number_of_columns_in_one_simulation = 11
time_sample_for_data = 0.01
resolution = 2

### Convert output data from simulation to proper model input

In [ ]:
logger.info("Converting time columns from data time to float")
train = simulation_data_time_to_float(simulation_data, number_of_columns_in_one_simulation)
logger.info("Converting tacho columns from impulses to angular velocity")
train = add_angular_velocity_calculated_from_pulses(train, number_of_columns_in_one_simulation, 
                                                    time_sample_for_data,
                                                    resolution)
logger.info("Reshape and prepare data to train")
X,_ = prepare_data_to_train(train, number_of_columns_in_one_simulation)

## Initialize model
Inside of **`ModelCasingFault`** all trained multimodels are initialize. Name of models to load is specified by parameter. So if you use `LSTM` as parameter then prediction will be perform on LSTM model which had best accuracy on test data while trained. Allowed models are: `LSTM`, `LSTM_FCN`, `FCNPlus`

In [ ]:
model = ModelCasingFault('LSTM')

## Model prediction
Based on the processed data, a classification is made
There are 8 different classes

- **0 - no failure**
- **1 - failures: sensor drift**
- **2 - failures: shaft wear**
- **3 - failures: sensor drift, shaft wear**
- **4 - failures: gear tooth fault****
- **5 - failures: gear tooth fault, sensor drift**
- **6 - failures: gear tooth fault, shaft wear**
- **7 - failures: gear tooth fault, shaft wear, sensor drift**

In [3]:
model.get_preds(X)

4
